In [1]:
#-*- coding:utf-8 -*-
import random
import string
import pickle
import os
import numpy as np
import pandas as pd
import tensorflow as tf

max_name_length = 6

vocabulary_list = []
vocabulary = {}

vocab = None
model = None
train_x, train_x_vec, train_y = None, None, None
vaild_x, vaild_x_vec, vaild_y = None, None, None


def read_file(filename):
    x, y = [], []
    with open(filename, 'r', encoding='utf-8') as f:
        first_line = True
        for line in f:
            if first_line is True:
                first_line = False
                continue
            sample = line.strip().split(',')
            if len(sample) == 2 and sample[0] is not None and len(sample[0]) > 0:
                x.append(sample[0])
                if sample[1] == '0':
                    y.append([1, 0])  # 男
                else:
                    y.append([0, 1])  # 女
    return x, y

def read_data():
    global train_x, train_x_vec, train_y
    global vaild_x, vaild_x_vec, vaild_Y
    train_x, train_y = read_file('data.csv')
#     vaild_x, vaild_y = read_file('vaild.csv')
#     train_x = train_x + vaild_x
#     train_y = train_y + vaild_y
#     data = pd.read_csv('name.csv', encoding='utf-8').values
#     train_x, train_y = data[:,0:1], data[:,1:]
#     train_x = np.array(train_x, dtype=np.object)
#     train_x = train_x.flatten()
#     train_y = np.array(train_y, dtype=np.int32)
#     train_y = to_categorical(train_y, 2)
#     data = pd.read_csv('csv.csv', encoding='utf-8').values
#     vaild_x, vaild_y = data[:,0:1], data[:,1:]
#     vaild_x = np.array(vaild_x, dtype=np.object)
#     vaild_x = vaild_x.flatten()
#     vaild_y = np.array(vaild_y, dtype=np.int32)
#     vaild_y = to_categorical(vaild_y, 2)


def build_cab():
    global vocab, vocabulary_list, vocabulary
    word_map = "word.map"
    if os.path.exists(word_map):
        with open(word_map, mode='rb') as fp:  
            vocab = pickle.loads(fp.read()) 
    else:
        counter = 0
        for name in train_x:
            if hasattr(name, '__iter__'):
                counter += 1
                tokens = [word for word in name]
                for word in tokens:
                    if word in vocabulary:
                        vocabulary[word] += 1
                    else:
                        vocabulary[word] = 1
            else:
                print("wtf data??")
#         for name in vaild_x:
#             if hasattr(name, '__iter__'):
#                 counter += 1
#                 tokens = [word for word in name]
#                 for word in tokens:
#                     if word in vocabulary:
#                         vocabulary[word] += 1
#                     else:
#                         vocabulary[word] = 1
#             else:
#                 print("wtf data??")
        vocabulary_list = [' '] + sorted(vocabulary, key=vocabulary.get, reverse=True)
        vocab = dict([(x, y) for (y, x) in enumerate(vocabulary_list)])
        data = pickle.dumps(obj=vocab)  
        with open(word_map, mode='wb') as fp:  
            fp.write(data)

def build_vec():
    global train_x_vec
    global vaild_x_vec
    train_x_vec = []
    for name in train_x:
        name_vec = []
        if hasattr(name, '__iter__'):
            for i in range(min(max_name_length, len(name))):
                name_vec.append(vocab.get(name[i]))
        while len(name_vec) < max_name_length:
            name_vec.append(0)
        train_x_vec.append(name_vec)
    train_x_vec = np.array(train_x_vec)
#     vaild_x_vec = []
#     for name in vaild_x:
#         name_vec = []
#         if hasattr(name, '__iter__'):
#             for word in name:
#                 name_vec.append(vocab.get(word))
#         while len(name_vec) < max_name_length:
#             name_vec.append(0)
#         vaild_x_vec.append(name_vec)
#     vaild_x_vec = np.array(vaild_x_vec)


read_data()
build_cab()
build_vec()

In [2]:
output = None
input_size = max_name_length
num_classes = 2

batch_size = 0
num_batch = 0 

X = tf.placeholder(tf.int32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, num_classes])

dropout_keep_prob = tf.placeholder(tf.float32)

# 算法:Convolutional Neural Network For Sentence Classification<Yoon Kim>
def neural_network(vocabulary_size, embedding_size=128, num_filters=192):
    # embedding layer:lookup-table
    with tf.name_scope("embedding"):
        # W:给每个字一个128大小的均匀分布:{h:n,w:128} => 对于每个字,字典有128个解释,初始分布平均([1,-1]记录影响强弱)
        W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        # embedded_chars:获取字Xi对应的Wi:{h:5,w:128}
        embedded_chars = tf.nn.embedding_lookup(W, X) 
        # embedded_chars_expanded:展开分布获取通道:{h:5,w:128,c:1}
        embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    # filter_sizes:卷积核设计
    # Q:为什么不设2? A:2只是多了一类AB的关联,而姓名和第一个字关联不大
    filter_sizes = [i for i in range(3, input_size+1)]
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            # filter_shape:卷积核形状:{h:[3,4,5],w:128,in_c:1,out_c:256}
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            # W:构造填充正太分布的卷积核:{h:[3,4,5],w:128,in_c:1,out_c:256} => 字典的解释影响概率
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
            # b:biases:添加使其变成非线性激活的常量:{l:256}
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]))
            # conv:卷积:{h:[3,2,1],w:128,c:192} => 将每相邻长度[3,4,5]字的解释糅杂一起
            conv = tf.nn.conv2d(embedded_chars_expanded, W, strides=[1, 1, 1, 1], padding="VALID")
            # h:relu激活:{h:[3,2,1],w:128}
            h = tf.nn.relu(tf.nn.bias_add(conv, b))
            # pooled:池化:窗口:{h:[3,2,1],w:1}:{h:[1,1,1],w:128} => 求出影响概率最大的解释
            pooled = tf.nn.max_pool(h, ksize=[1, input_size - filter_size + 1, 1, 1],
                                    strides=[1, 1, 1, 1], padding='VALID')
            pooled_outputs.append(pooled)

    # num_filters_total:256*3=576
    num_filters_total = num_filters * len(filter_sizes)
    # h_pool_flat:所有解释影响概率扁平化:{l:576}
    h_pool = tf.concat(pooled_outputs, 3)
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
    with tf.name_scope("dropout"):
        h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)
    with tf.name_scope("output"):
        W = tf.get_variable("W", shape=[num_filters_total, num_classes], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
        output = tf.nn.xw_plus_b(h_drop, W, b)
        
    return output

output = neural_network(len(vocab))

In [3]:
len(train_x_vec), len(train_y), len(vocab), len(vocabulary_list)

(1211482, 1211482, 6974, 0)

In [4]:
batch_size = 13461
num_batch = len(train_x_vec) // batch_size
optimizer = tf.train.AdamOptimizer(1e-3)
# 计算loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=Y))
# 计算梯度
grads_and_vars = optimizer.compute_gradients(loss)
# 把梯度应用到变量
train_op = optimizer.apply_gradients(grads_and_vars)

saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(201):
        for i in range(num_batch):
            batch_x = train_x_vec[i*batch_size : (i+1)*batch_size]
            batch_y = train_y[i*batch_size : (i+1)*batch_size]
            loss_ = sess.run([train_op, loss], feed_dict={X:batch_x, Y:batch_y, dropout_keep_prob:0.5})
            #print(e, i, loss_)
        print(e, loss_)
        
        if e % 100 == 0:
                saver.save(sess, "./model/gender.ckpt", global_step=e)
    save_path = saver.save(sess, "./model/gender.ckpt")
    print("Model saved in file: %s" % save_path)

In [5]:
x = []
name_list = ["刘倩妤", "陈浩民", "邓子琪", "林鹭攸", "别说话吻我", "麻辣香菇", "麻辣香锅", 
             "蔡文姬", "孙尚香", "狄仁杰", "李元芳", "东王太一", "荆轲", "阿柯",
             "陈奕迅", "方皓玟", "宋黛霆", "张信哲", "徐佳莹", "李玉刚", "叶蒨文", "陈一发儿"]
for name in name_list:
    name_vec = []
    if hasattr(name, '__iter__'):
        for word in name:
            if vocab.get(word) != None:
                name_vec.append(vocab.get(word))
        while len(name_vec) < max_name_length:
            name_vec.append(0)
        x.append(name_vec)
print(x)
x = np.array(x)

[[4, 230, 810, 0, 0, 0], [5, 84, 183, 0, 0, 0], [154, 58, 176, 0, 0, 0], [10, 1411, 1872, 0, 0, 0], [1640, 2836, 3170, 2733, 2830, 0], [499, 6303, 299, 3518, 0, 0], [499, 6303, 299, 4438, 0, 0], [112, 7, 444, 0, 0, 0], [21, 270, 299, 0, 0, 0], [995, 356, 33, 0, 0, 0], [3, 290, 53, 0, 0, 0], [39, 1, 748, 137, 0, 0], [576, 1559, 0, 0, 0, 0], [770, 425, 0, 0, 0, 0], [5, 418, 1139, 0, 0, 0], [88, 366, 1348, 0, 0, 0], [69, 1707, 1142, 0, 0, 0], [2, 624, 257, 0, 0, 0], [19, 46, 221, 0, 0, 0], [3, 27, 104, 0, 0, 0], [87, 3767, 7, 0, 0, 0], [5, 137, 438, 742, 0, 0]]


In [6]:
saver = tf.train.Saver(tf.global_variables())
with tf.Session() as sess:
    # 恢复前一次训练
    ckpt = tf.train.get_checkpoint_state('./model/')
    if ckpt != None:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print("没找到模型")
        
    res = sess.run(output, {X:x, dropout_keep_prob:1.0})

#     i = 0
#     for name in name_list:
#         print(name, res[i])
#         i += 1
    predictions = tf.argmax(output, 1)
    res = sess.run(predictions, {X:x, dropout_keep_prob:0.5})

    i = 0
    for name in name_list:
        print(name, '女' if res[i] == 1 else '男')
        i += 1

./model/gender.ckpt
INFO:tensorflow:Restoring parameters from ./model/gender.ckpt
刘倩妤 女
陈浩民 男
邓子琪 女
林鹭攸 女
别说话吻我 女
麻辣香菇 女
麻辣香锅 女
蔡文姬 女
孙尚香 女
狄仁杰 男
李元芳 女
东王太一 男
荆轲 男
阿柯 男
陈奕迅 男
方皓玟 女
宋黛霆 男
张信哲 男
徐佳莹 女
李玉刚 男
叶蒨文 女
陈一发儿 男


In [ ]:
# 继续训练
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state('./model/')
    if ckpt != None:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        for e in range(501, 801):
            for i in range(num_batch):
                batch_x = train_x_vec[i*batch_size : (i+1)*batch_size]
                batch_y = train_y[i*batch_size : (i+1)*batch_size]
                loss_ = sess.run([train_op, loss], feed_dict={X:batch_x, Y:batch_y, dropout_keep_prob:0.5})
                #print(e, i, loss_)
            print(e, loss_)
            if e % 100 == 0:
                    saver.save(sess, "./model/gender.ckpt", global_step=e)
        save_path = saver.save(sess, "./model/gender.ckpt")
        print("Model saved in file: %s" % save_path)

In [7]:
# 训练集准确率计算
acc = 0
saver = tf.train.Saver(tf.global_variables())
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state('./model/')
    if ckpt != None:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print("没找到模型")
        
    predictions = tf.argmax(output, 1)
    for i in range(num_batch):
        batch_x = train_x_vec[i*batch_size : (i+1)*batch_size]
        batch_y = train_y[i*batch_size : (i+1)*batch_size]
        res = sess.run(predictions, feed_dict={X:batch_x, dropout_keep_prob:1.0})

        for j in range(len(res)):
            k = i*batch_size+j
            if train_y[k][res[j]]:
                acc = acc + 1
            else:
                pass
#                print(train_x[k], '女' if res[j] == 1 else '男', '女' if train_y[k][1] else '男')

./model/gender.ckpt
INFO:tensorflow:Restoring parameters from ./model/gender.ckpt


In [8]:
acc, len(train_y), acc/len(train_y)

(1084493, 1211482, 0.8951787975388821)

In [ ]:
for i in range(len(train_x)):
    for j in train_x[i]:
        if j == ' ':
            print(i)
            
print("End")